In [113]:
!pip install ollama
!pip install -U transformers accelerate bitsandbytes pandas

In [114]:
instruction1 = {
    "query_rewriter": 
"""
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are an expert in evaluating AI assistants for Indian e-commerce. 
IMPORTANT: Ensure that the final JSON response contains ALL SEVEN FIELDS in the following format. 
DO NOT deviate from this structure, regardless of the examples mentioned in the prompt. 

Assess the AI's response using these criteria:

Criteria:
1. Query Continuation (1-5)
Is the query a continuation of the previous conversation?
Rate (1-5): 5 = correct, 1 = incorrect.


2. Query Refinement (1-5)
Continuation: Assess context use.
New Query: Check clarity and grammar.
Rate (1-5): 5 = excellent, 1 = poor.


3. Response Accuracy
Check for accuracy, relevance, and instruction adherence.

Steps:
Step 1: Analyze Input

The information that you will be evaluating is given between ``` and ```

```
[[tool_type]]: {tool_type}
[[sys_prompt]]: {sys_prompt}
[[human_prompt]]: {human_prompt}
[[response]]: {response}
```

Step 2: Query Continuation

Determine continuation.
Rate (1-5).

Step 3: Query Refinement

Rate refinement or clarity (1-5).

Step 4: Response Accuracy

Step 5: Improved Response

Provide improved response if needed. If sufficient, state "NA".

IMPORTANT:
Ensure that the final JSON response contains ALL SEVEN FIELDS in the following format. 
DO NOT deviate from this structure, regardless of the examples mentioned in the prompt. 
DO NOT BLINDLY FOLLOW THE EXAMPLES, THE EXAMPLES ARE MEANT TO HELP YOU GENERATE THE FINAL JSON RESPONSES IN THE BELOW FORMAT.
The final goal is to produce the response in this specified format:

{{
"query": "The user's input query",
"query_rewriter_explanation": "Overall quality of the response",
"is_continuation_rating": "Rating (1-5)",
"is_continuation_explanation": "Explanation for the rating",
"refined_query_rating": "Rating (1-5)",
"refined_query_explanation": "Explanation for the rating",
"improved_response": "Improved response or 'NA'"
}}

HERE ARE SOME EXAMPLES FOR YOUR UNDERSTANDING ON HOW YOU SHOULD EVALUATE THE USER QUERIES AND RETURN THE CORRESPONDING JSON RESPONSES:
            
            EXAMPLE 1 :
            Previous Conversation History:
            "thought": "The user is asking for assistance with tracking an order, which is related to the previous conversation about the assistant's capabilities. Therefore, this is a continuation of the previous conversation, and the conversation history needs to be used to capture the user's intent.",
            "is_continuation": true,
            "query": "I also need socks to go with them."
            FINAL JSON Response:
            {{
              "query": "I also need socks to go with them.",
              "query_rewriter_explanation": "The response continues the conversation about purchasing running gear, specifically adding socks.",
              "is_continuation_rating": "5",
              "is_continuation_explanation": "The query logically follows the previous conversation about running shoes.",
              "refined_query_rating": "5",
              "refined_query_explanation": "The query is clear and well-structured.",
              "improved_response": "NA"
            }}
            
            EXAMPLE 2 : 
            Previous Conversation History:
            "thought": "The user is exploring options for budget smartphones, with an emphasis on specific features like camera quality and battery life. This is part of an ongoing conversation about electronic gadgets.",
            "is_continuation": true,
            "query": "What about tablets?"
            FINAL JSON Response:
            {{
              "query": "What about tablets?",
              "query_rewriter_explanation": "The response shifts the conversation from smartphones to tablets.",
              "is_continuation_rating": "5",
              "is_continuation_explanation": "While related to electronics, it changes the product category from smartphones to tablets.",
              "refined_query_rating": "4",
              "refined_query_explanation": "The query is clear but lacks specific features or requirements.",
              "improved_response": "Can you suggest some budget tablets with a good camera and long battery life?"
            }}

            EXAMPLE 3 : 
            Previous Conversation History:
            "thought": "The user is preparing for a wedding and is seeking fashion advice, specifically looking for a dress that is elegant and pastel-colored. The conversation now extends to finding matching accessories.",
            "is_continuation": true,
            "query": "What accessories would go well with it?"
            FINAL JSON Response:
            {{
              "query": "What accessories would go well with it?",
              "query_rewriter_explanation": "The response appropriately continues the conversation about wedding attire by adding a query about accessories.",
              "is_continuation_rating": "5",
              "is_continuation_explanation": "The query is a natural progression in the context of shopping for a wedding outfit.",
              "refined_query_rating": "5",
              "refined_query_explanation": "The query is clear and relevant.",
              "improved_response": "NA"
            }}
            
            EXAMPLE 4 : 
            Previous Conversation History:
            "thought": "The user has expressed interest in cooking, specifically Italian cuisine, and is now broadening their interest to include baking.",
            "is_continuation": true,
            "query": "I also want to learn baking."
            FINAL JSON Response:
            {{
              "query": "I also want to learn baking.",
              "query_rewriter_explanation": "The response expands the user's interest in cooking to include baking.",
              "is_continuation_rating": "5",
              "is_continuation_explanation": "The query is related but introduces a new subcategory.",
              "refined_query_rating": "5",
              "refined_query_explanation": "The query is clear and concise.",
              "improved_response": "Can you also suggest some books on baking?"
            }}
            
            EXAMPLE 5 :
            Previous Conversation History:
            "thought": "The user is interested in buying a laptop for gaming and video editing, and the conversation now includes a query about compatible monitors.",
            "is_continuation": true,
            "query": "What about monitors?"
            FINAL JSON Response:
            {{
              "query": "What about monitors?",
              "query_rewriter_explanation": "The response changes the focus from laptops to monitors.",
              "is_continuation_rating": "5",
              "is_continuation_explanation": "The query introduces a new product category.",
              "refined_query_rating": "4",
              "refined_query_explanation": "The query is clear but would benefit from additional details.",
              "improved_response": "Can you suggest some budget monitors that are suitable for gaming and video editing?"
            }}
            
            EXAMPLE 6 :
            Previous Conversation History:
            "thought": "The user previously inquired about the return policy for purchased items. This new query, however, shifts to a completely unrelated topic about grocery shopping.",
            "is_continuation": false,
            "query": "What are the best deals on groceries?"
            FINAL JSON Response:
            {{
              "query": "What are the best deals on groceries?",
              "query_rewriter_explanation": "The response does not follow the previous conversation about return policies and instead introduces a new topic about grocery shopping.",
              "is_continuation_rating": "5",
              "is_continuation_explanation": "The query shifts the topic from return policies to grocery deals, which is unrelated.",
              "refined_query_rating": "5",
              "refined_query_explanation": "The query is clear and well-phrased for the new topic.",
              "improved_response": "NA"
            }}

IMPORTANT: Now that you understand the expected format and structure of the final output, create the final JSON response based on the following information: [[response]]: {response}

Evaluation Response in JSON:

<|eot_id|>
<|end_of_text|>
"""
}

In [115]:
instruction2 = {
    "router":
"""
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are an expert evaluator of AI assistants and chatbots for online INDIAN retail e-commerce domain. Your goal is to assess the performance of an AI assistant's response using the following criteria: relevance, accuracy, and specificity. Rate the response from 1 (worst) to 5 (best).
Here's how you should approach the assessment:
Step 1: Analyze the Prompt: Carefully study the user query, app_context, page_context, and any available disambiguation_context. Understand the supported app categories and tools. Verify if the response follows instructions and guidelines from the prompts.
Step 2. Evaluate Routing Decisions by assessing the following parameters:

thought: Does it precisely explain the reasoning behind the category selection?
status: Is the message shown to the end-user concise (less than 10 words), relevant, and sets a clear expectation while the answer is being prepared?
tool_name: Is the chosen tool the most appropriate for the determined category and user intent?
tool_input: Are the input fields populated correctly? Does the provided information allow the target tool to effectively address the user's query?
app_name: Is the specified app a valid provider for the chosen category?
category: Is the selected category unequivocally correct from the below mentioned list of supported categories?

Note that the below, if the right value is identified as 'category', it will get mapped to the left value as there are ONLY 6 supported categories by the application - 'electronics', 'fashion', 'grocery', 'pharmacy', 'labs' and 'finance'. The mapping below is validated and YOU must adhere to it.
Example: If the category identified was "household essentials" it will get mapped to "grocery".

"grocery": "household essentials",
"grocery": "kitchen supplies",
"grocery": "cleaning supplies",
"grocery": "baby_supplies",
"grocery": "daily hygiene",
"grocery": "stationery",
"grocery": "pet_supplies",
"grocery": "nutritional remedies"
"electronics": "smartphones",
"electronics": "home entertainment",
"electronics": "computers",
"electronics": "software",
"electronics": "video games",
"electronics": "home appliances",
"electronics": "kitchen appliances"
"pharmacy": "pharmaceutical remedies",
"pharmacy": "skin care",
"pharmacy": "hair care",
"pharmacy": "womens_health"
"fashion": "cosmetic enhancements",
"fashion": "furniture",
"fashion": "sports",
"fashion": "jewellery",
"fashion": "books",
"fashion": "footwear",
"fashion": "fitness",
"fashion": "art_supplies",
"fashion": "toys",
"fashion": "grooming essentials"
"finance": "flights",
"finance": "hotels"
"labs": "labs",
"labs": "lab tests"

IMPORTANT:
Based on the mentioned criteria and guidelines, assess 'status', 'tool_name', 'app_name', 'category' independently. Utilize this explanation to rate 'status', 'tool_name', 'app_name', 'category' individually. Also, make sure to offer a more appropriate JSON response for the query with the specified parameters ONLY in case of sub-par responses.

The information that you will be evaluating is given between ``` and ```

```
[[tool_type]]: {tool_type}
[[sys_prompt]]: {sys_prompt}
[[human_prompt]]: {human_prompt}
[[response]]: {response}
```

IMPORTANT:
Ensure that the final JSON response contains ALL TWELVE FIELDS in the following format. 
DO NOT deviate from this structure, regardless of the examples mentioned in the prompt. 
The final goal is to produce the response in this specified format:

query: the user input
overall_rating: the rating of the response between 1 and 5
router_explanation: your expert opinion on the overall quality of the response
status_rating: rating of the message shown to the end-user between 1 and 5
status_explanation: your explanation for the status
tool_name_rating: rating of the tool_name selection between 1 and 5
tool_name_explanation: your explanation for the tool_name
app_name_rating: rating of the app_name selection between 1 and 5
app_name_explanation: explanation for the app_name
category_rating: rating of the category selection between 1 and 5
category_explanation: explanation for the category
improved_response: the improved JSON response

Evaluation Response in JSON:

<|eot_id|>
<|end_of_text|>
"""
}

In [116]:
instruction3 = {
    "single_step_tool":
"""
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are an expert evaluator of AI assistants and chatbots for online INDIAN retail e-commerce domain. You have an in-depth understanding of the retail e-commerce domain with precision and detail. Your expertise in retail e-commerce is going to be crucial for this evaluation task. Your GOAL is to evaluate and rate the quality of the response on a scale of (1-5) based on the following criteria:
Criteria:

Helpfulness: Does the response answer the user's query directly?
Conciseness: Is the response clear and focused?
Accuracy: Is the information provided factually correct?
Level of Detail: Does the response provide adequate information without being overwhelming?

Scale:

5 (Excellent): Exceeds expectations in all aspects. Offers highly relevant, specific, helpful, and actionable guidance to the user.
4 (Good): A strong response, fulfilling the user's needs well. May have very minor shortcomings.
3 (Average): A satisfactory response, but perhaps lacking some specificity, or offering suggestions that are not fully aligned with the user's intent.
2 (Below Average): Contains significant mismatches with the user's query or lacks basic helpfulness.
1 (Poor): Fails to understand the user's query or generates unhelpful/ irrelevant suggestions.

Here's how you MUST approach the assessment:
Step 1: Verify the response's adherence to the instructions.
Step 2: Start by succinctly highlighting the positive aspects of the response, followed by the negative aspects and areas for improvement. Use this analysis to assign a rating on a scale of 1 to 5.
Step 3: When evaluating 'search_term', prioritize #Specificity# as a key factor as it helps search engines refine results to be more relevant to the user. It is essential to balance user-friendliness and search engine compatibility when assessing both 'search_term' and 'suggestions', as they directly impact the end-user experience.
Note: DO NOT consider scenarios where users could have provided additional information to influence the evaluation.
Below, you will find guidelines, rules, and examples of both GOOD and POOR 'search_term' choices to guide you in evaluating 'search_term':
i) GOOD:

Concrete product names, brands, categories, or features (examples: "Nike Air Max shoes", "organic cotton t-shirt", "waterproof hiking boots")
Includes terms readily mappable to product attributes in inventory and have a greater chance of directly  matching product names. Consider how the search terms map to product categories, attributes, or descriptions.
Example#1: Sunscreen for dry skin.
GOOD search_term: "hydrating sunscreen"
Why it's GOOD: Clear Product Focus, it centers on sunscreen as the core product. "Hydrating" directly addresses a common need for those with dry skin, making the search more specific than general "sunscreen" queries."Hydrating" is a common attribute found in product descriptions, making it easy for search engines to match relevant items from your inventory.
Rating: 5
Example#2: Ingredients for pesto pasta
GOOD search_term: "pasta"
Why it's GOOD: Valid search term, it is a key ingredient for making pesto pasta recipe and is also specific and effective for the user providing him with different kinds of pasta types. The 'search_term' 'pasta' is both specific and concise, making it an effective term for search engines. It directly maps to a product category that can yield a variety of relevant results, helping the user to find the exact type of pasta they need for the recipe.
Rating: 5
Example#3: Help me find a good gift for my 15 year old girl who just completed high school
Good search_term: "jewelry"
Why it's GOOD: "Jewelry" is a reasonably specific product category, likely to yield helpful results for the user. It aligns well with common graduation gifts and would appeal to many teenage girls.
Rating: 4
Example:#4: What tech gadget would impress my gadget-loving friend?
Good search_term: "VR headset"
Why it's GOOD:  VR headsets are a well-defined product category. This specific search_term would produce far more targeted results than a broader term like "tech gadget." The query explicitly states the friend is a "gadget-lover." VR headsets are cutting-edge technology, making them a likely hit with someone interested in tech.
Rating: 5

ii) POOR:

POOR search_terms have broad terms or prepositions or contain heavy reliance on subjective descriptions such as "for" ,"stylish", "best", "cheap", "in", "and", "with" etc. They are VERY BAD for the search engine.
Contains subjective terms ("best", "stylish", etc) in favor of concrete features or characteristics.
Contains multiple search_terms like "t-shirt, jeans" in "search_term" field.
Example#1: high protein fruits
POOR search_term: "high-protein fruits"
Why it's POOR: Extremely broad, lacks any indication of a product name or category.
Rating: 1
Example#2: food for increasing joint lubrication
POOR search_term: "joint health food"
Why it's POOR: Despite "joint health food" having sub par specificity. It cannot be effectively handled by a search engine and lacks direct mapping to inventory.
Rating: 1
Example#3: medicine for cough and cold
POOR search_term: "cough and cold medicine"
Why it's POOR: "cough and cold medicine" is not the right kind of specificity, a search engine can yield results for. A user likely wouldn't search "cough and cold" medicines in a search bar in a retail e-commerce apps for over-the-counter medication.
Rating: 1
Example#4: sunscreen for oily skin
POOR search_term: "sunscreen for oily skin"
Why it's POOR:  It uses a broad term ("sunscreen") and relies on a user-centric descriptor ("for oily skin"). While understandable to humans, search engines might struggle to differentiate between "sunscreen for oily skin" and other types of sunscreen.
Rating: 3
Example#5: Help me find a good gift for my 15 year old girl who just completed high school
POOR search_term: "15 year gift sets"
Why it's POOR:  The search_term is too generic. "Gift sets" is a broad category, and the focus on the age ("15 year") doesn't sufficiently narrow the options.
Rating: 2

IMPORTANT:

DO NOT assume or include extraneous information to search_term from a user standpoint.
For lower quality responses, offer a more appropriate JSON response for the end user's query while keeping the context intact. I'll tip you $200 for every good improved response
Having "stylish", "best", "cheap", "for", "in", "and" etc in 'search_term' field are VERY BAD for the search engine and should be scored low.
DO NOT consider cases where user could have provided additional or more information in your evaluation.
The AI assistant SHOULD NOT offer any medical advise and should avoid suggesting prescription medications.

The information that you will be evaluating is given between ``` and ```
```
[[tool_type]]: {tool_type}
[[sys_prompt]]: {sys_prompt}
[[human_prompt]]: {human_prompt}
[[response]]: {response}
```

IMPORTANT:
Ensure that the final JSON response contains ALL TEN FIELDS in the following format. 
DO NOT deviate from this structure, regardless of the examples mentioned in the prompt. 
The final goal is to produce the response in this specified format:

query: the user input
overall_rating: the rating of the overall quality of response between 1 and 5
single_step_tool_explanation: your expert opinion on the overall quality of the response.
search_term_rating: rating between 1 and 5, if present or reply "NA" if not available.
search_term_explanation: your analysis of the 'search_term' along with clear justifications for your ratings, addressing strengths, weaknesses, and improvements.
suggestions_rating: rating of the suggestions between 1 and 5.
suggestions_explanation: your analysis of the 'suggestions' along with clear justifications for your ratings, addressing strengths, weaknesses, and improvements.
message_rating: rating of the message spoken to the end-user between 1 and 5.
message_explanation: your analysis of the 'message', along with clear justifications for your ratings, addressing strengths, weaknesses, and improvements.
improved_response: the improved JSON response

Evaluation Response in JSON:

<|eot_id|>
<|end_of_text|>
"""
}

In [117]:
import pandas as pd
import json
import ollama

file_path = '/Users/slanglabs/Downloads/FINALLY4.csv'
df = pd.read_csv(file_path)

tool_type_columns = {
    "query_rewriter": ["sys_prompt", "human_prompt", "response"],
    "router": ["sys_prompt", "human_prompt", "response"],
    "single_step_tool": ["sys_prompt", "human_prompt", "response"]
}

evaluation_responses = []

for index, row in df.iterrows():
    tool_type = row['tool_type']

    if tool_type in tool_type_columns:
        
        response = {
            "tool_type": tool_type,
            "sys_prompt": row.get('sys_prompt', ""),
            "human_prompt": row.get('human_prompt', ""),
            "response": row.get('response', "")
        }

        evaluation_responses.append(response)
evaluation_responses_json = json.dumps(evaluation_responses, indent=4)
print(evaluation_responses[0])
# print(evaluation_responses_json[:1000] + "\n... (output truncated) ...\n" + evaluation_responses_json[-1000:])

{'tool_type': 'query_rewriter', 'sys_prompt': 'You are a smart capable assistant for Tata Neu, which is an app in the e-commerce space.', 'human_prompt': 'You are an expert at understanding and refining user input based on the conversation history. This task is extremely crucial for the assistant. Based on the previous conversation history and the new user input, generate a refined query that captures the full and precise intent of the user\'s current query.\n\nUse the JSON schema defined below to guide your output. Do not include ANYTHING other than the JSON response in your output.\n{\n  "properties": {\n    "thought": {\n      "description": "write your thoughts on whether the current user query is related to a previous conversation and if the conversation history is to be used to write the new query",\n      "type": "string"\n    },\n    "is_continuation": {\n    "description": "boolean flag to signify whether is the current user input is continuation of previous conversation",\n  

In [118]:
import gc

final_outputs = []

for idx, response in enumerate(evaluation_responses):
    element = response
    tool_type = element['tool_type']

    if tool_type == "router":
        final = instruction2["router"].format(
            tool_type=element["tool_type"],
            response=element["response"],
            sys_prompt=element["sys_prompt"],
            human_prompt=element["human_prompt"]
        )
    elif tool_type == "query_rewriter":
        final = instruction1["query_rewriter"].format(
            tool_type=element["tool_type"],
            response=element["response"],
            sys_prompt=element["sys_prompt"],
            human_prompt=element["human_prompt"]
        )
    elif tool_type == "single_step_tool":
        final = instruction3["single_step_tool"].format(
            tool_type=element["tool_type"],
            response=element["response"],
            sys_prompt=element["sys_prompt"],
            human_prompt=element["human_prompt"]
        )

    final_outputs.append(final)
print(final_outputs[0])
# print(len(final_outputs))


<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are an expert in evaluating AI assistants for Indian e-commerce. 
IMPORTANT: Ensure that the final JSON response contains ALL SEVEN FIELDS in the following format. 
DO NOT deviate from this structure, regardless of the examples mentioned in the prompt. 

Assess the AI's response using these criteria:

Criteria:
1. Query Continuation (1-5)
Is the query a continuation of the previous conversation?
Rate (1-5): 5 = correct, 1 = incorrect.


2. Query Refinement (1-5)
Continuation: Assess context use.
New Query: Check clarity and grammar.
Rate (1-5): 5 = excellent, 1 = poor.


3. Response Accuracy
Check for accuracy, relevance, and instruction adherence.

Steps:
Step 1: Analyze Input

The information that you will be evaluating is given between ``` and ```

```
[[tool_type]]: query_rewriter
[[sys_prompt]]: You are a smart capable assistant for Tata Neu, which is an app in the e-commerce space.
[[human_prompt]]: You are an expe

In [120]:
import pandas as pd
import json
import ollama
import time
from datetime import datetime, timezone, timedelta

def call_tool(prompt, tool_type):
    tools = []

    if tool_type == "router":
        tools = [
            {
                'type': 'function',
                'function': {
                    'name': f'{tool_type}',
                    'description': """
                                   Evaluate the AI assistant's response for relevance, accuracy, and specificity in the Indian retail e-commerce context. 
                                   Rate each aspect (1-5) and provide an improved JSON response if necessary, ensuring adherence to specified categories and criteria.
                                   IMPORTANT! ENSURE THAT THE FINAL JSON RESPONSE HAS ALL TWELVE FIELDS.
                                   """,
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'prompt': {
                                'type': 'string',
                                'description': f'{prompt}IMPORTANT! ENSURE THAT THE FINAL JSON RESPONSE HAS ALL TWELVE FIELDS.',
                            },
                        },
                        'required': ['prompt'],
                    },
                },
            }
        ]
    elif tool_type == "query_rewriter":
        tools = [
            {
                'type': 'function',
                'function': {
                    'name': f'{tool_type}',
                    'description': """
                                   Evaluate the AI’s response for an Indian e-commerce context based on query continuation, refinement, and accuracy. 
                                   Rate each aspect (1-5) and provide an improved response if necessary, using the provided JSON format.
                                   IMPORTANT! ENSURE THAT THE FINAL JSON RESPONSE HAS ALL SEVEN FIELDS.
                                    """,                    
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'prompt': {
                                'type': 'string',
                                'description': f'{prompt}IMPORTANT! ENSURE THAT THE FINAL JSON RESPONSE HAS ALL SEVEN FIELDS.',
                            },
                        },
                        'required': ['prompt'],
                    },
                },
            }
        ]
    elif tool_type == "single_step_tool":
        tools = [
            {
                'type': 'function',
                'function': {
                    'name': f'{tool_type}',
                    'description': """
                                   Evaluate the AI assistant's response in the Indian retail e-commerce context for helpfulness, conciseness, accuracy, and level of detail. 
                                   Rate each aspect (1-5) and provide an improved JSON response if necessary. 
                                   Focus on specific, user-friendly search terms and avoid subjective or broad terms.
                                   IMPORTANT! ENSURE THAT THE FINAL JSON RESPONSE HAS ALL TEN FIELDS.
                                   """,
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'prompt': {
                                'type': 'string',
                                'description': f'{prompt}IMPORTANT! ENSURE THAT THE FINAL JSON RESPONSE HAS ALL TEN FIELDS.',
                            },
                        },
                        'required': ['prompt'],
                    },
                },
            }
        ]

    response = ollama.chat(
        model='llama3.1:8b-instruct-q4_0',
        messages=[{'role': 'user', 'content': prompt}],
        tools=tools,
        stream=False,
        format='json',
    )

    return response['message']

start_index = 0
end_index = min(4, len(final_outputs))

start_time = time.time()
start_time_ist = datetime.now(timezone.utc) + timedelta(hours=5, minutes=30)
start_time_ist_str = start_time_ist.strftime('%Y-%m-%d %H:%M:%S %Z')
print(f"Start time (IST): {start_time_ist_str}")

for idx in range(start_index, end_index):
    element = final_outputs[idx]
    tool_type = evaluation_responses[idx]['tool_type']
    response = call_tool(element, tool_type)
    result = response.get('tool_calls', response) 
    print(f"Raw result for index {idx}:\n{result['content']}\n")

    try:
        json_response_str = json.dumps(result, indent=4)
    except json.JSONDecodeError as e:
        print(f"JSONDecodeError at index {idx}: {e}")
        json_response_str = str(result)

    df.at[idx, 'Llama-3-8b Scores'] = json_response_str

    df.to_csv(file_path, index=False)

end_time = time.time()

elapsed_time = end_time - start_time
end_time_ist = datetime.now(timezone.utc) + timedelta(hours=5, minutes=30)
end_time_ist_str = end_time_ist.strftime('%Y-%m-%d %H:%M:%S %Z')

print(f"Total time taken: {elapsed_time} seconds")
print(f"End time (IST): {end_time_ist_str}")

Start time (IST): 2024-07-29 17:25:06 UTC
Raw result for index 0:
{
  "query": "sure",
  "query_rewriter_explanation": "The response does not follow the previous conversation about pasta recipes and instead introduces a new topic.",
  "is_continuation_rating": "5",
  "is_continuation_explanation": "The query shifts the topic from cooking to a general acknowledgement, which is unrelated.",
  "refined_query_rating": "5",
  "refined_query_explanation": "The query is clear and well-phrased for the new topic.",
  "improved_response": "NA"
}

Raw result for index 1:
{
    "query": "sure",
    "overall_rating": 3,
    "router_explanation": "The user's response 'sure' is a casual conversation or affirmation, but it doesn't provide any specific context for the conversation.",
    "status_rating": 4,
    "status_explanation": "The status of the conversation is considered good because the user has responded with a positive affirmation.",
    "tool_name_rating": 5,
    "tool_name_explanation": "Th